# PID控制理论和算法

## 是个什么东西？

在实际工程中，应用最为广泛的调节器控制规律为比例(**P**)、积分(**I**)、微分(**D**)控制，简称PID控制，又称PID调节。

PID控制器连续系统中技术最为成熟、应心最为广泛的一种控制算法。**说白了就是一种调控算法。比如常见的无人机的飞行姿态和飞行速度调节。**

那么什么情况下会用到PID调节？ 

当被控对象的结构和参数不能完全掌握，或得不到精确的数学模型时，控制理论的其他计算难以采用时，系统控制器的结构和参数必须依靠经验和现场调试来确定，这个时候应用PID控制技术最为方便。


![1](./images/1.png)

上图是PID的框架图，可以看输出反馈到输入， 输入分别通过**P**, **I**, **D**计算后，求和后再输出。同时反馈到输入，如此闭环控制。

对应的数学模型就下公式：

![2](./images/2.png)

公式中, Kp--比例增益
      
     Tt--积分时间常数
     
     TD--微分时间常数
     
     u(t)---PID控制器的输出信号
     
     e(t)---给定值r(t)与测量值的差

## 简单的水箱液位模型

#### 比例模型：

假设我们面对的系统是一个简单的水箱液位问题，要从空箱开始注水到达某个高度，而你能控制的变量就是注水龙头的开关大小，这个简单的数学模型就是：**dx=u**.

对应这个系统，我们只需要一个比例环节**u = Kpe** 就能将其控制住。

此时, Kp的大小代表了水龙头的粗细，越粗调的越快，也就是所谓的“调大比例系数一般会加快系统响应”。如下图：

![3-1](./images/3-1.jpeg)

#### 积分模型：

假设这个水箱不仅仅是装水的容器，还需要持续稳定的给用户供水，这个系统的数学模型就需要增加一项： **dx=u-c**, 这里的c是个正的常数。

我们发现如果系统控制器只有一个比例环节，那么当系统稳定，也就是**dx=0**的时候（**dx=u-c** >> **dx=Kpe-c** >> **Kpe = c**),恰好**e=c/kp**, 在系统稳定时不为0， 液位离我们想要的高度总是差那么一点点，这也就是所谓的稳态误差，或者叫静差。

这个时候c是固定的，那么当然Kp越大， e就越小。 这也就是所谓的增大比例系数P在有静差的情况下有利于减小静差。如下图：

![3-2](./images/3-2.jpeg)

从上面的公式**e=c/kp**可以看出, kp再大也不可能把e变成0，然后就有人想到了， 相比于与第一次的模型，这里只多了一个漏水的窟窿。它漏多少我给它补多少， 那不就成了之前的简单系统了么！ 那么靠谁来补呢？ 这个是积分环节就派上用场了。

我们把之前的控制器变成了比例环节+积分环节：

![3-3](./images/3-3.png)

积分环节的意义就是相当于你增加了一个水龙头，这个水龙头的开关规则是水位比预定高度低就一直往大了拧，比预想高度高就往小了拧。如果漏水速度不变，那么总有一天这个水龙头出水的速度恰好和漏水的速度相等了。这个时候系统就和之前的意义了，而且，这个时候静差也就没有了。这就是所谓积分环节可以消除系统静差。

什么叫积分时间常数呢？ 一般PID控制里，表示积分环节敏感度的那个系数中，Ti就积分时间常数。从上公式中可以看出， 积分时间常数越大，积分环节系数就越小，积分环节就越不敏感（也就是第二个水龙头越细）。

当只有一个比例环节的水龙头注水的时候，是不会注水注多的，因为离的越近水龙头关的越小。

但是当用两个水龙头注水的时候， 在没到预期高度前第二个积分环节的水笼头可以一直往大了拧，当到达预定高度的时候它恰好拧到了最大，自然而然就会注水注多了。而多出去的这部分水叫做“超调”。这个水龙头越粗，注的水就会越多， 它调到恰好等于漏水速度的时间就会越快。

于是，老师告诉我们增大积分时间，有利于减小超调，减小震荡，使系统的稳定性增加，但是系统静差消除时间变长。如下图：

![3-4](./images/3-4.jpeg)

接下来看点有意思的东西，还是上面的这个系统，假如我们选用相同的积分时间常数，但是选择不同的比例系统会如何呢？

![3-5](./images/3-5.jpeg)

看到上图这副图，一些记性好的 童鞋可能会有疑问。因为老师明明说过“过大的比例系数会使系统有比较大的超调，并产生震荡，使稳定性变坏”，但是上面这幅图里怎么比例大的反而超调小呢？

其实上面这幅图很好解释，我们说明了PI控制器超调出现原因是积分这个水龙头在到达目标液位时也恰好开到了最大。而比例这个水龙头越粗，那么它在超出目标液位时对超调的抑制也就越明显。

#### 微分模型：(??看不懂这块)

在上面的系统中，我们假设用户用水是一个固定值，但实际情况中用户的用水量往往是变化的，假如我们的系统是**dx=u-c(t)** 呢？

来分析一下：

我们的控制目标是让**x=xd**, 系统误差的定义是： **e=xd-x**。（即系统误差为0）

那么误差状态方程就是：

![3-6](./images/3-6.svg)

上面我们设定的控制目标是个常数， 所有 **dxd = 0**, 那么

![3-7](./images/3-7.svg)

从上面的这个式子我们可以看出， 当**e=0**时候，积分不在变化， 而c(t)是始终在变化。

那么此时de不恒为零， 也就是说e不恒为零。也就是说，当c变成了c(t)的时候，e=0就不再是系统的稳定平衡点了，经典意义上系统不在稳定。

这里加入一个微分环节D变成PID控制会不会让系统重新稳定呢？

![3-8](./images/3-8.svg)

当加入微分环节：

![3-9](./images/3-9.svg)

那么对于任何T(T > 0), 微分环节都让de的变化减慢了，也就是说 **"微分环节主要的作用是在响应过程中抑制偏差向任何方向的变化"， "微分常数不能过大，否则会使响应过程提前制动，延长调节时间"**。

微分作用不能消除静差，单独使用意义不大，一般需要与比例、积分控制作用配合使用，构成PD或PID控制。